In [1]:
import pandas as pd
import re
import math

In [2]:
reg_season_data = pd.read_csv('data/2022_data//MRegularSeasonDetailedResults.csv')
team_ids = pd.read_csv('data/2022_data/MTeams.csv')
tourney_seeds = pd.read_csv('data/2022_data/MNCAATourneySeeds.csv')
team_names = pd.read_csv('data/2022_data/MTeams.csv')
conferences = pd.read_csv('data/2022_data/MTeamConferences.csv')

In [3]:
recent_regular_season_data = reg_season_data[reg_season_data['Season'] >= 2010]

In [ ]:
recent_regular_season_data[recent_regular_season_data['Season'] == 2021]

# Getting Regular Season Data

In [4]:
mean_data = []
for team_id in team_ids['TeamID']:
    sum_data = []
    count_data = []
    team_reg_season_data = recent_regular_season_data[(recent_regular_season_data['WTeamID'] == team_id) | (recent_regular_season_data['LTeamID'] == team_id)]
    winning_data = team_reg_season_data[team_reg_season_data['WTeamID'] == team_id][['Season', 'WScore',
        'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
    winning_data['TeamID'] = team_id
    winning_data.columns = winning_data.columns.str.replace('W', '')
    winning_data_sums = winning_data.groupby(['Season', 'TeamID']).sum()
    winning_data_counts = winning_data.groupby(['Season', 'TeamID']).count()
    sum_data.append(winning_data_sums)
    count_data.append(winning_data_counts)
    
    
    losing_data = team_reg_season_data[team_reg_season_data['WTeamID'] == team_id][['Season', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]
    losing_data['TeamID'] = team_id
    losing_data.columns = losing_data.columns.str.replace('L', '')
    losing_data_sums = losing_data.groupby(['Season', 'TeamID']).sum()
    losing_data_counts = losing_data.groupby(['Season', 'TeamID']).count()
    sum_data.append(losing_data_sums)
    count_data.append(losing_data_counts)
    team_sums = pd.concat(sum_data).groupby(['Season', 'TeamID']).sum()
    team_counts = pd.concat(count_data).groupby(['Season', 'TeamID']).sum()
    team_means = team_sums / team_counts
    mean_data.append(team_means)


In [5]:
team_reg_season_average_data = pd.concat(mean_data)
team_reg_season_average_data = team_reg_season_average_data.add_prefix('Season_Avg_')

In [ ]:
team_reg_season_average_data

In [6]:
tournament_data = pd.read_csv('data/2022_data/MNCAATourneyDetailedResults.csv')
seed_data = pd.read_csv('data/2022_data/MNCAATourneySeeds.csv')

In [7]:
tourney_data = []

for team_id in team_ids['TeamID']:
    team_data = []
    winning_team_tourney_data = tournament_data[tournament_data['WTeamID'] == team_id][['Season', 'WTeamID', 'WScore']]
    winning_team_tourney_data.columns = ['Season', 'TeamID', 'Score']
    winning_team_tourney_data = winning_team_tourney_data.merge(team_reg_season_average_data, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'])
    tourney_data.append(winning_team_tourney_data)
    
    
    
    losing_team_tourney_data = tournament_data[tournament_data['LTeamID'] == team_id][['Season', 'LTeamID', 'LScore']]
    losing_team_tourney_data.columns = ['Season', 'TeamID', 'Score']

    losing_team_tourney_data = losing_team_tourney_data.merge(team_reg_season_average_data, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'])
    tourney_data.append(losing_team_tourney_data)
    

In [8]:
tourney_model_data = pd.concat(tourney_data)

In [9]:
tourney_model_data.sort_values('Season', inplace=True)

In [10]:
tourney_model_data = tourney_model_data.merge(seed_data, on=['Season', 'TeamID']).drop_duplicates()

In [11]:
tourney_model_data['Seed'] = tourney_model_data['Seed'].map(lambda x: int(''.join(re.findall('[0-9]', x))))

# Linear Regression Model

In [ ]:
X = tourney_model_data.drop(['Season','TeamID', 'Score'], axis=1)

In [ ]:
y = tourney_model_data['Score']

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import operator


In [23]:
def remove_most_insignificant(df, results):
    # use operator to find the key which belongs to the maximum value in the dictionary:
    max_p_value = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    # this is the feature you want to drop:
    df.drop(columns = max_p_value, inplace = True)
    return df

In [ ]:
insignificant_feature = True
while insignificant_feature:
        model = sm.OLS(y, X)
        results = model.fit()
        significant = [p_value < 0.05 for p_value in results.pvalues]
        if all(significant):
            insignificant_feature = False
        else:
            if X.shape[1] == 1:  # if there's only one insignificant variable left
                print('No significant features found')
                results = None
                insignificant_feature = False
            else:            
                X = remove_most_insignificant(X, results)
print(results.summary())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

# Logistic Regression

In [12]:
log_tourney_data = tournament_data[['Season', 'WTeamID', 'LTeamID']]



In [13]:
log_tourney_data = log_tourney_data[log_tourney_data['Season'] != 2021]

In [14]:
team1_tourney_data = log_tourney_data.merge(team_reg_season_average_data, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])

team1_tourney_data = team1_tourney_data.merge(team_reg_season_average_data, left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'], suffixes=['', '_Opponent'])

team1_tourney_data = team1_tourney_data.merge(seed_data, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])

team1_tourney_data = team1_tourney_data.merge(seed_data, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], suffixes=['', '_Opponent'])

team1_tourney_data['Seed'] = team1_tourney_data['Seed'].map(lambda x: int(''.join(re.findall('[0-9]', x))))
team1_tourney_data['Seed_Opponent'] = team1_tourney_data['Seed_Opponent'].map(lambda x: int(''.join(re.findall('[0-9]', x))))

team1_tourney_data['Seed_Diff'] = team1_tourney_data['Seed_Opponent'] - team1_tourney_data['Seed']



team1_tourney_data['Win'] = 1

In [15]:
team2_tourney_data = log_tourney_data.merge(team_reg_season_average_data, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])

team2_tourney_data = team2_tourney_data.merge(team_reg_season_average_data, left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'], suffixes=['_Opponent', ''])

team2_tourney_data = team2_tourney_data.merge(seed_data, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])

team2_tourney_data = team2_tourney_data.merge(seed_data, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], suffixes=['_Opponent', ''])

team2_tourney_data['Seed'] = team2_tourney_data['Seed'].map(lambda x: int(''.join(re.findall('[0-9]', x))))
team2_tourney_data['Seed_Opponent'] = team2_tourney_data['Seed_Opponent'].map(lambda x: int(''.join(re.findall('[0-9]', x))))

team2_tourney_data['Seed_Diff'] = team2_tourney_data['Seed_Opponent'] - team2_tourney_data['Seed']

team2_tourney_data['Win'] = 0

In [16]:
combined_tourney_data = pd.concat([team1_tourney_data, team2_tourney_data])

In [17]:
combined_tourney_data = combined_tourney_data[['Season_Avg_Score', 'Season_Avg_FGM',
       'Season_Avg_FGA', 'Season_Avg_FGM3', 'Season_Avg_FGA3',
       'Season_Avg_FTM', 'Season_Avg_FTA', 'Season_Avg_OR', 'Season_Avg_DR',
       'Season_Avg_Ast', 'Season_Avg_TO', 'Season_Avg_Stl', 'Season_Avg_Blk',
       'Season_Avg_PF', 'Season_Avg_Score_Opponent', 'Season_Avg_FGM_Opponent',
       'Season_Avg_FGA_Opponent', 'Season_Avg_FGM3_Opponent',
       'Season_Avg_FGA3_Opponent', 'Season_Avg_FTM_Opponent',
       'Season_Avg_FTA_Opponent', 'Season_Avg_OR_Opponent',
       'Season_Avg_DR_Opponent', 'Season_Avg_Ast_Opponent',
       'Season_Avg_TO_Opponent', 'Season_Avg_Stl_Opponent',
       'Season_Avg_Blk_Opponent', 'Season_Avg_PF_Opponent', 'Seed',
       'Seed_Opponent', 'Seed_Diff', 'Win']]

In [18]:
X = combined_tourney_data.drop(['Win', 'Seed', 'Seed_Opponent'], axis=1)

In [19]:
y = combined_tourney_data['Win']

In [24]:
insignificant_feature = True
while insignificant_feature:
        model = sm.OLS(y, X)
        results = model.fit()
        significant = [p_value < 0.05 for p_value in results.pvalues]
        if all(significant):
            insignificant_feature = False
        else:
            if X.shape[1] == 1:  # if there's only one insignificant variable left
                print('No significant features found')
                results = None
                insignificant_feature = False
            else:            
                X = remove_most_insignificant(X, results)
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                    Win   R-squared (uncentered):                   0.615
Model:                            OLS   Adj. R-squared (uncentered):              0.613
Method:                 Least Squares   F-statistic:                              265.1
Date:                Mon, 14 Mar 2022   Prob (F-statistic):                   1.03e-268
Time:                        20:32:01   Log-Likelihood:                         -793.40
No. Observations:                1334   AIC:                                      1603.
Df Residuals:                    1326   BIC:                                      1644.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [25]:
logistic = LogisticRegression().fit(X, y)


In [ ]:
logistic.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, clf.predict(X_test))

In [26]:
def predict_matchup(team1, team2, season, team1_seed, team2_seed, model):
    team1_name = list(team_names[team_names['TeamID'] == team1]['TeamName'])[0]
    team2_name = list(team_names[team_names['TeamID'] == team2]['TeamName'])[0]
    print(f'{team1_name} vs {team2_name}')
    season_data = team_reg_season_average_data.query(f'Season == {season}').reset_index()
    team1_data = season_data.query(f'TeamID == {team1}')
    team2_data = season_data.query(f'TeamID == {team2}')
#     team1_data = team1_data.merge(seed_data, on=['Season','TeamID'])
#     team2_data = team2_data.merge(seed_data, on=['Season','TeamID'])

    match_data = team1_data.merge(team2_data, on='Season', suffixes=['', '_Opponent'])
    
#     match_data['Seed'] = match_data['Seed'].map(lambda x: int(''.join(re.findall('[0-9]', x))))
#     match_data['Seed_Opponent'] = match_data['Seed_Opponent'].map(lambda x: int(''.join(re.findall('[0-9]', x))))
    
    match_data['Seed_Diff'] = team2_seed - team1_seed
    
    match_data = match_data[['Season_Avg_Score', 'Season_Avg_FGA', 'Season_Avg_DR', 'Season_Avg_FGA_Opponent', 'Season_Avg_FTM_Opponent', 'Season_Avg_DR_Opponent', 'Season_Avg_Ast_Opponent', 'Seed_Diff']]

    win_prob = model.predict_proba(match_data)[0][1]
    
    team1_win_prob = win_prob*100
    team2_win_prob = 100 - team1_win_prob
    if win_prob > 0.5:
        team1_odds = math.ceil((100*team1_win_prob)/(team1_win_prob - 100))
        team2_odds = -math.ceil(((100*(team2_win_prob-100))/(team2_win_prob)))
    elif win_prob < 0.5:
        team1_odds = -math.ceil(((100*(team1_win_prob-100))/(team1_win_prob)))
        team2_odds = math.ceil((100*team2_win_prob)/(team2_win_prob - 100))

    else:
        team1_odds = 100
        team2_odds = 100

    
    print(f'{team1_name} has a {round(win_prob*100,2)}% Chance of Winning ({team1_odds})')
    print(f'{team2_name} has a {round((1- win_prob)*100,2)}% Chance of Winning ({team2_odds})')
    
    predictions = {'ID': f'2022_{team1}_{team2}', 'Pred': win_prob}
    return predictions
    


In [ ]:
matchups_2021 = tournament_data[['Season', 'WTeamID', 'LTeamID']]
matchups_2021 = matchups_2021[matchups_2021['Season'] == 2021]

In [ ]:
for index, matchup in matchups_2021.iterrows():
    predict_matchup(matchup['WTeamID'], matchup['LTeamID'], matchup['Season'])

# Decision Tree

In [ ]:
from sklearn import tree


In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

# Random Forest

In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf = GridSearchCV(rf, {'n_estimators': range(100,1000,100)})

In [ ]:
rf = rf.fit(X_train, y_train)

In [ ]:
rf.best_estimator_
rf.score(X_test, y_test)

# Gradient Tree Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf = GridSearchCV(clf, {'n_estimators': range(100,1000,100), 'max_depth': range(1,10)})

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
clf.score(X_test, y_test)

# KNN

In [27]:
from sklearn.neighbors import KNeighborsClassifier


In [31]:
parameters = {'n_neighbors': range(1,100,2)}
neigh = KNeighborsClassifier(n_neighbors=3)
neigh = GridSearchCV(neigh, parameters)


In [32]:
neigh = neigh.fit(X, y)
neigh.best_estimator_

KNeighborsClassifier(n_neighbors=53)

In [ ]:
neigh.score(X_test, y_test)

# Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

# 2022 Predictions 

In [ ]:
predictions = []

In [ ]:
prediction = predict_matchup(1260, 1326, 2022, 10, 7)

In [ ]:
predictions.append(prediction)

In [ ]:
pd.DataFrame(predictions).to_csv('2022_march_madness_swtn_logistic_predictions.csv', index=False)

In [ ]:
predictions = pd.read_csv('2022_march_madness_swtn_logistic_predictions.csv')

In [ ]:
predictions

In [33]:
seeds_2022 = pd.read_csv('data/2022_data/MNCAATourneySeeds.csv')

In [34]:
tourney_teams = list(seeds_2022[seeds_2022['Season'] == 2022]['TeamID'])

In [35]:
import itertools

In [132]:
predict_matchup(1116, 1181, 2022, 4, 2, logistic)

Arkansas vs Duke
Arkansas has a 29.01% Chance of Winning (244)
Duke has a 70.99% Chance of Winning (-244)


{'ID': '2022_1116_1181', 'Pred': 0.29012355305591764}

In [36]:
predictions = []
for matchup in list(itertools.combinations(tourney_teams, 2)):
    team1_seed = seeds_2022[(seeds_2022['TeamID'] == matchup[0]) & (seeds_2022['Season'] == 2022)]['Seed'].values[0]
    team2_seed = seeds_2022[(seeds_2022['TeamID'] == matchup[1]) & (seeds_2022['Season'] == 2022)]['Seed'].values[0]
    team1_seed = int(''.join(re.findall('[0-9]', team1_seed)))
    team2_seed = int(''.join(re.findall('[0-9]', team2_seed)))

    if matchup[0] > matchup[1]:

        predictions.append(predict_matchup(matchup[1], matchup[0], 2022, team2_seed, team1_seed, logistic))
    else:
        predictions.append(predict_matchup(matchup[0], matchup[1], 2022, team1_seed, team2_seed, logistic))

    

Baylor vs Kentucky
Baylor has a 50.91% Chance of Winning (-103)
Kentucky has a 49.09% Chance of Winning (103)
Baylor vs Purdue
Baylor has a 66.9% Chance of Winning (-202)
Purdue has a 33.1% Chance of Winning (202)
Baylor vs UCLA
Baylor has a 67.54% Chance of Winning (-208)
UCLA has a 32.46% Chance of Winning (208)
Baylor vs St Mary's CA
Baylor has a 62.11% Chance of Winning (-163)
St Mary's CA has a 37.89% Chance of Winning (163)
Baylor vs Texas
Baylor has a 68.74% Chance of Winning (-219)
Texas has a 31.26% Chance of Winning (219)
Baylor vs Murray St
Baylor has a 72.2% Chance of Winning (-259)
Murray St has a 27.8% Chance of Winning (259)
Baylor vs North Carolina
Baylor has a 84.0% Chance of Winning (-525)
North Carolina has a 16.0% Chance of Winning (525)
Baylor vs Marquette
Baylor has a 90.5% Chance of Winning (-952)
Marquette has a 9.5% Chance of Winning (952)
Baylor vs San Francisco
Baylor has a 85.45% Chance of Winning (-587)
San Francisco has a 14.55% Chance of Winning (587)
Bay

Duke has a 38.93% Chance of Winning (156)
Kentucky has a 61.07% Chance of Winning (-156)
Kentucky vs Texas Tech
Kentucky has a 61.9% Chance of Winning (-162)
Texas Tech has a 38.1% Chance of Winning (162)
Arkansas vs Kentucky
Arkansas has a 21.95% Chance of Winning (355)
Kentucky has a 78.05% Chance of Winning (-355)
Connecticut vs Kentucky
Connecticut has a 29.23% Chance of Winning (242)
Kentucky has a 70.77% Chance of Winning (-242)
Alabama vs Kentucky
Alabama has a 23.0% Chance of Winning (334)
Kentucky has a 77.0% Chance of Winning (-334)
Kentucky vs Michigan St
Kentucky has a 81.63% Chance of Winning (-444)
Michigan St has a 18.37% Chance of Winning (444)
Boise St vs Kentucky
Boise St has a 19.61% Chance of Winning (409)
Kentucky has a 80.39% Chance of Winning (-409)
Kentucky vs Memphis
Kentucky has a 79.93% Chance of Winning (-398)
Memphis has a 20.07% Chance of Winning (398)
Davidson vs Kentucky
Davidson has a 9.16% Chance of Winning (991)
Kentucky has a 90.84% Chance of Winning

Georgia St vs Purdue
Georgia St has a 16.43% Chance of Winning (508)
Purdue has a 83.57% Chance of Winning (-508)
Kansas vs Purdue
Kansas has a 54.83% Chance of Winning (-121)
Purdue has a 45.17% Chance of Winning (121)
Auburn vs Purdue
Auburn has a 51.73% Chance of Winning (-107)
Purdue has a 48.27% Chance of Winning (107)
Purdue vs Wisconsin
Purdue has a 51.18% Chance of Winning (-104)
Wisconsin has a 48.82% Chance of Winning (104)
Providence vs Purdue
Providence has a 42.5% Chance of Winning (135)
Purdue has a 57.5% Chance of Winning (-135)
Iowa vs Purdue
Iowa has a 41.04% Chance of Winning (143)
Purdue has a 58.96% Chance of Winning (-143)
LSU vs Purdue
LSU has a 36.62% Chance of Winning (173)
Purdue has a 63.38% Chance of Winning (-173)
Purdue vs USC
Purdue has a 63.38% Chance of Winning (-173)
USC has a 36.62% Chance of Winning (173)
Purdue vs San Diego St
Purdue has a 63.11% Chance of Winning (-171)
San Diego St has a 36.89% Chance of Winning (171)
Creighton vs Purdue
Creighton 

TAM C. Christi vs UCLA
TAM C. Christi has a 10.14% Chance of Winning (885)
UCLA has a 89.86% Chance of Winning (-885)
TX Southern vs UCLA
TX Southern has a 13.66% Chance of Winning (631)
UCLA has a 86.34% Chance of Winning (-631)
Arizona vs UCLA
Arizona has a 54.43% Chance of Winning (-119)
UCLA has a 45.57% Chance of Winning (119)
UCLA vs Villanova
UCLA has a 35.01% Chance of Winning (185)
Villanova has a 64.99% Chance of Winning (-185)
Tennessee vs UCLA
Tennessee has a 51.45% Chance of Winning (-105)
UCLA has a 48.55% Chance of Winning (105)
Illinois vs UCLA
Illinois has a 45.32% Chance of Winning (120)
UCLA has a 54.68% Chance of Winning (-120)
Houston vs UCLA
Houston has a 47.68% Chance of Winning (109)
UCLA has a 52.32% Chance of Winning (-109)
Colorado St vs UCLA
Colorado St has a 28.93% Chance of Winning (245)
UCLA has a 71.07% Chance of Winning (-245)
Ohio St vs UCLA
Ohio St has a 30.15% Chance of Winning (231)
UCLA has a 69.85% Chance of Winning (-231)
Seton Hall vs UCLA
Seton

Delaware has a 11.67% Chance of Winning (757)
St Mary's CA has a 88.33% Chance of Winning (-757)
Bryant vs St Mary's CA
Bryant has a 11.75% Chance of Winning (751)
St Mary's CA has a 88.25% Chance of Winning (-751)
St Mary's CA vs Wright St
St Mary's CA has a 69.34% Chance of Winning (-226)
Wright St has a 30.66% Chance of Winning (226)
Murray St vs Texas
Murray St has a 38.07% Chance of Winning (162)
Texas has a 61.93% Chance of Winning (-162)
North Carolina vs Texas
North Carolina has a 26.34% Chance of Winning (279)
Texas has a 73.66% Chance of Winning (-279)
Marquette vs Texas
Marquette has a 23.15% Chance of Winning (332)
Texas has a 76.85% Chance of Winning (-332)
San Francisco vs Texas
San Francisco has a 20.06% Chance of Winning (398)
Texas has a 79.94% Chance of Winning (-398)
Texas vs Virginia Tech
Texas has a 61.97% Chance of Winning (-162)
Virginia Tech has a 38.03% Chance of Winning (162)
Indiana vs Texas
Indiana has a 14.76% Chance of Winning (577)
Texas has a 85.24% Chan

...

Bryant has a 24.03% Chance of Winning (316)
San Diego St has a 75.97% Chance of Winning (-316)
San Diego St vs Wright St
San Diego St has a 60.57% Chance of Winning (-153)
Wright St has a 39.43% Chance of Winning (153)
Creighton vs Miami FL
Creighton has a 53.22% Chance of Winning (-113)
Miami FL has a 46.78% Chance of Winning (113)
Creighton vs Iowa St
Creighton has a 57.73% Chance of Winning (-136)
Iowa St has a 42.27% Chance of Winning (136)
Creighton vs Richmond
Creighton has a 68.97% Chance of Winning (-222)
Richmond has a 31.03% Chance of Winning (222)
Creighton vs S Dakota St
Creighton has a 63.89% Chance of Winning (-176)
S Dakota St has a 36.11% Chance of Winning (176)
Colgate vs Creighton
Colgate has a 21.93% Chance of Winning (355)
Creighton has a 78.07% Chance of Winning (-355)
Creighton vs Jacksonville St
Creighton has a 68.66% Chance of Winning (-219)
Jacksonville St has a 31.34% Chance of Winning (219)
Creighton vs TAM C. Christi
Creighton has a 82.44% Chance of Winning 

Illinois vs Richmond
Illinois has a 81.62% Chance of Winning (-443)
Richmond has a 18.38% Chance of Winning (443)
Houston vs Richmond
Houston has a 83.0% Chance of Winning (-488)
Richmond has a 17.0% Chance of Winning (488)
Colorado St vs Richmond
Colorado St has a 68.56% Chance of Winning (-218)
Richmond has a 31.44% Chance of Winning (218)
Ohio St vs Richmond
Ohio St has a 69.81% Chance of Winning (-231)
Richmond has a 30.19% Chance of Winning (231)
Richmond vs Seton Hall
Richmond has a 21.46% Chance of Winning (365)
Seton Hall has a 78.54% Chance of Winning (-365)
Richmond vs TCU
Richmond has a 23.15% Chance of Winning (332)
TCU has a 76.85% Chance of Winning (-332)
Loyola-Chicago vs Richmond
Loyola-Chicago has a 57.24% Chance of Winning (-133)
Richmond has a 42.76% Chance of Winning (133)
Michigan vs Richmond
Michigan has a 63.21% Chance of Winning (-171)
Richmond has a 36.79% Chance of Winning (171)
Richmond vs UAB
Richmond has a 28.31% Chance of Winning (253)
UAB has a 71.69% Cha

TAM C. Christi has a 17.49% Chance of Winning (471)
TCU has a 82.51% Chance of Winning (-471)
Loyola-Chicago vs TAM C. Christi
Loyola-Chicago has a 73.87% Chance of Winning (-282)
TAM C. Christi has a 26.13% Chance of Winning (282)
Michigan vs TAM C. Christi
Michigan has a 78.4% Chance of Winning (-362)
TAM C. Christi has a 21.6% Chance of Winning (362)
TAM C. Christi vs UAB
TAM C. Christi has a 21.75% Chance of Winning (359)
UAB has a 78.25% Chance of Winning (-359)
Chattanooga vs TAM C. Christi
Chattanooga has a 78.76% Chance of Winning (-370)
TAM C. Christi has a 21.24% Chance of Winning (370)
Longwood vs TAM C. Christi
Longwood has a 74.47% Chance of Winning (-291)
TAM C. Christi has a 25.53% Chance of Winning (291)
Delaware vs TAM C. Christi
Delaware has a 65.48% Chance of Winning (-189)
TAM C. Christi has a 34.52% Chance of Winning (189)
Bryant vs TAM C. Christi
Bryant has a 65.66% Chance of Winning (-191)
TAM C. Christi has a 34.34% Chance of Winning (191)
TAM C. Christi vs Wrig

Illinois vs UAB
Illinois has a 67.11% Chance of Winning (-204)
UAB has a 32.89% Chance of Winning (204)
Chattanooga vs Illinois
Chattanooga has a 25.27% Chance of Winning (295)
Illinois has a 74.73% Chance of Winning (-295)
Illinois vs Longwood
Illinois has a 69.64% Chance of Winning (-229)
Longwood has a 30.36% Chance of Winning (229)
Delaware vs Illinois
Delaware has a 14.74% Chance of Winning (578)
Illinois has a 85.26% Chance of Winning (-578)
Bryant vs Illinois
Bryant has a 14.85% Chance of Winning (573)
Illinois has a 85.15% Chance of Winning (-573)
Illinois vs Wright St
Illinois has a 71.0% Chance of Winning (-244)
Wright St has a 29.0% Chance of Winning (244)
Colorado St vs Houston
Colorado St has a 34.07% Chance of Winning (193)
Houston has a 65.93% Chance of Winning (-193)
Houston vs Ohio St
Houston has a 60.6% Chance of Winning (-153)
Ohio St has a 39.4% Chance of Winning (153)
Houston vs Seton Hall
Houston has a 60.82% Chance of Winning (-155)
Seton Hall has a 39.18% Chance

Delaware vs Longwood
Delaware has a 31.7% Chance of Winning (215)
Longwood has a 68.3% Chance of Winning (-215)
Bryant vs Longwood
Bryant has a 31.87% Chance of Winning (213)
Longwood has a 68.13% Chance of Winning (-213)
Longwood vs Wright St
Longwood has a 43.23% Chance of Winning (131)
Wright St has a 56.77% Chance of Winning (-131)
Bryant vs Delaware
Bryant has a 42.67% Chance of Winning (134)
Delaware has a 57.33% Chance of Winning (-134)
Delaware vs Wright St
Delaware has a 33.12% Chance of Winning (201)
Wright St has a 66.88% Chance of Winning (-201)
Bryant vs Wright St
Bryant has a 33.3% Chance of Winning (200)
Wright St has a 66.7% Chance of Winning (-200)


In [37]:
prediction_df = pd.DataFrame(predictions)

In [ ]:
len(prediction_df)

In [39]:
prediction_df.to_csv('2022_bracket_logistic_predictions_v2.csv', index=False)

In [40]:
predictions = []
for matchup in list(itertools.combinations(tourney_teams, 2)):
    team1_seed = seeds_2022[(seeds_2022['TeamID'] == matchup[0]) & (seeds_2022['Season'] == 2022)]['Seed'].values[0]
    team2_seed = seeds_2022[(seeds_2022['TeamID'] == matchup[1]) & (seeds_2022['Season'] == 2022)]['Seed'].values[0]
    team1_seed = int(''.join(re.findall('[0-9]', team1_seed)))
    team2_seed = int(''.join(re.findall('[0-9]', team2_seed)))

    if matchup[0] > matchup[1]:

        predictions.append(predict_matchup(matchup[1], matchup[0], 2022, team2_seed, team1_seed, neigh))
    else:
        predictions.append(predict_matchup(matchup[0], matchup[1], 2022, team1_seed, team2_seed, neigh))

    

Baylor vs Kentucky
Baylor has a 54.72% Chance of Winning (-120)
Kentucky has a 45.28% Chance of Winning (120)
Baylor vs Purdue
Baylor has a 50.94% Chance of Winning (-103)
Purdue has a 49.06% Chance of Winning (103)
Baylor vs UCLA
Baylor has a 64.15% Chance of Winning (-178)
UCLA has a 35.85% Chance of Winning (178)
Baylor vs St Mary's CA
Baylor has a 66.04% Chance of Winning (-194)
St Mary's CA has a 33.96% Chance of Winning (194)
Baylor vs Texas
Baylor has a 60.38% Chance of Winning (-152)
Texas has a 39.62% Chance of Winning (152)
Baylor vs Murray St
Baylor has a 66.04% Chance of Winning (-194)
Murray St has a 33.96% Chance of Winning (194)
Baylor vs North Carolina
Baylor has a 71.7% Chance of Winning (-253)
North Carolina has a 28.3% Chance of Winning (253)
Baylor vs Marquette
Baylor has a 77.36% Chance of Winning (-341)
Marquette has a 22.64% Chance of Winning (341)
Baylor vs San Francisco
Baylor has a 86.79% Chance of Winning (-657)
San Francisco has a 13.21% Chance of Winning (6

Kentucky has a 81.13% Chance of Winning (-430)
Wyoming has a 18.87% Chance of Winning (430)
Akron vs Kentucky
Akron has a 13.21% Chance of Winning (657)
Kentucky has a 86.79% Chance of Winning (-657)
Kentucky vs Yale
Kentucky has a 88.68% Chance of Winning (-783)
Yale has a 11.32% Chance of Winning (783)
Kentucky vs St Peter's
Kentucky has a 84.91% Chance of Winning (-562)
St Peter's has a 15.09% Chance of Winning (562)
Kentucky vs Norfolk St
Kentucky has a 88.68% Chance of Winning (-783)
Norfolk St has a 11.32% Chance of Winning (783)
Gonzaga vs Kentucky
Gonzaga has a 47.17% Chance of Winning (112)
Kentucky has a 52.83% Chance of Winning (-112)
Duke vs Kentucky
Duke has a 47.17% Chance of Winning (112)
Kentucky has a 52.83% Chance of Winning (-112)
Kentucky vs Texas Tech
Kentucky has a 39.62% Chance of Winning (152)
Texas Tech has a 60.38% Chance of Winning (-152)
Arkansas vs Kentucky
Arkansas has a 41.51% Chance of Winning (140)
Kentucky has a 58.49% Chance of Winning (-140)
Connecti

Davidson has a 26.42% Chance of Winning (278)
Purdue has a 73.58% Chance of Winning (-278)
Notre Dame vs Purdue
Notre Dame has a 20.75% Chance of Winning (381)
Purdue has a 79.25% Chance of Winning (-381)
Purdue vs Rutgers
Purdue has a 79.25% Chance of Winning (-381)
Rutgers has a 20.75% Chance of Winning (381)
New Mexico St vs Purdue
New Mexico St has a 26.42% Chance of Winning (278)
Purdue has a 73.58% Chance of Winning (-278)
Purdue vs Vermont
Purdue has a 75.47% Chance of Winning (-307)
Vermont has a 24.53% Chance of Winning (307)
Montana St vs Purdue
Montana St has a 16.98% Chance of Winning (488)
Purdue has a 83.02% Chance of Winning (-488)
CS Fullerton vs Purdue
CS Fullerton has a 13.21% Chance of Winning (657)
Purdue has a 86.79% Chance of Winning (-657)
Georgia St vs Purdue
Georgia St has a 15.09% Chance of Winning (562)
Purdue has a 84.91% Chance of Winning (-562)
Kansas vs Purdue
Kansas has a 52.83% Chance of Winning (-112)
Purdue has a 47.17% Chance of Winning (112)
Auburn 

UCLA has a 58.49% Chance of Winning (-140)
USC has a 41.51% Chance of Winning (140)
San Diego St vs UCLA
San Diego St has a 28.3% Chance of Winning (253)
UCLA has a 71.7% Chance of Winning (-253)
Creighton vs UCLA
Creighton has a 30.19% Chance of Winning (231)
UCLA has a 69.81% Chance of Winning (-231)
Miami FL vs UCLA
Miami FL has a 37.74% Chance of Winning (165)
UCLA has a 62.26% Chance of Winning (-165)
Iowa St vs UCLA
Iowa St has a 32.08% Chance of Winning (211)
UCLA has a 67.92% Chance of Winning (-211)
Richmond vs UCLA
Richmond has a 22.64% Chance of Winning (341)
UCLA has a 77.36% Chance of Winning (-341)
S Dakota St vs UCLA
S Dakota St has a 24.53% Chance of Winning (307)
UCLA has a 75.47% Chance of Winning (-307)
Colgate vs UCLA
Colgate has a 22.64% Chance of Winning (341)
UCLA has a 77.36% Chance of Winning (-341)
Jacksonville St vs UCLA
Jacksonville St has a 15.09% Chance of Winning (562)
UCLA has a 84.91% Chance of Winning (-562)
TAM C. Christi vs UCLA
TAM C. Christi has a 

Seton Hall has a 28.3% Chance of Winning (253)
St Mary's CA has a 71.7% Chance of Winning (-253)
St Mary's CA vs TCU
St Mary's CA has a 62.26% Chance of Winning (-165)
TCU has a 37.74% Chance of Winning (165)
Loyola-Chicago vs St Mary's CA
Loyola-Chicago has a 37.74% Chance of Winning (165)
St Mary's CA has a 62.26% Chance of Winning (-165)
Michigan vs St Mary's CA
Michigan has a 30.19% Chance of Winning (231)
St Mary's CA has a 69.81% Chance of Winning (-231)
St Mary's CA vs UAB
St Mary's CA has a 60.38% Chance of Winning (-152)
UAB has a 39.62% Chance of Winning (152)
Chattanooga vs St Mary's CA
Chattanooga has a 32.08% Chance of Winning (211)
St Mary's CA has a 67.92% Chance of Winning (-211)
Longwood vs St Mary's CA
Longwood has a 32.08% Chance of Winning (211)
St Mary's CA has a 67.92% Chance of Winning (-211)
Delaware vs St Mary's CA
Delaware has a 32.08% Chance of Winning (211)
St Mary's CA has a 67.92% Chance of Winning (-211)
Bryant vs St Mary's CA
Bryant has a 16.98% Chance o

...

Creighton has a 50.94% Chance of Winning (-103)
Iowa St has a 49.06% Chance of Winning (103)
Creighton vs Richmond
Creighton has a 50.94% Chance of Winning (-103)
Richmond has a 49.06% Chance of Winning (103)
Creighton vs S Dakota St
Creighton has a 60.38% Chance of Winning (-152)
S Dakota St has a 39.62% Chance of Winning (152)
Colgate vs Creighton
Colgate has a 24.53% Chance of Winning (307)
Creighton has a 75.47% Chance of Winning (-307)
Creighton vs Jacksonville St
Creighton has a 62.26% Chance of Winning (-165)
Jacksonville St has a 37.74% Chance of Winning (165)
Creighton vs TAM C. Christi
Creighton has a 64.15% Chance of Winning (-178)
TAM C. Christi has a 35.85% Chance of Winning (178)
Creighton vs TX Southern
Creighton has a 69.81% Chance of Winning (-231)
TX Southern has a 30.19% Chance of Winning (231)
Arizona vs Creighton
Arizona has a 81.13% Chance of Winning (-430)
Creighton has a 18.87% Chance of Winning (430)
Creighton vs Villanova
Creighton has a 32.08% Chance of Winni

Arizona has a 88.68% Chance of Winning (-783)
Richmond has a 11.32% Chance of Winning (783)
Richmond vs Villanova
Richmond has a 30.19% Chance of Winning (231)
Villanova has a 69.81% Chance of Winning (-231)
Richmond vs Tennessee
Richmond has a 24.53% Chance of Winning (307)
Tennessee has a 75.47% Chance of Winning (-307)
Illinois vs Richmond
Illinois has a 71.7% Chance of Winning (-253)
Richmond has a 28.3% Chance of Winning (253)
Houston vs Richmond
Houston has a 69.81% Chance of Winning (-231)
Richmond has a 30.19% Chance of Winning (231)
Colorado St vs Richmond
Colorado St has a 56.6% Chance of Winning (-130)
Richmond has a 43.4% Chance of Winning (130)
Ohio St vs Richmond
Ohio St has a 60.38% Chance of Winning (-152)
Richmond has a 39.62% Chance of Winning (152)
Richmond vs Seton Hall
Richmond has a 33.96% Chance of Winning (194)
Seton Hall has a 66.04% Chance of Winning (-194)
Richmond vs TCU
Richmond has a 35.85% Chance of Winning (178)
TCU has a 64.15% Chance of Winning (-178)


Jacksonville St has a 35.85% Chance of Winning (178)
Loyola-Chicago has a 64.15% Chance of Winning (-178)
Jacksonville St vs Michigan
Jacksonville St has a 33.96% Chance of Winning (194)
Michigan has a 66.04% Chance of Winning (-194)
Jacksonville St vs UAB
Jacksonville St has a 39.62% Chance of Winning (152)
UAB has a 60.38% Chance of Winning (-152)
Chattanooga vs Jacksonville St
Chattanooga has a 54.72% Chance of Winning (-120)
Jacksonville St has a 45.28% Chance of Winning (120)
Jacksonville St vs Longwood
Jacksonville St has a 49.06% Chance of Winning (103)
Longwood has a 50.94% Chance of Winning (-103)
Delaware vs Jacksonville St
Delaware has a 35.85% Chance of Winning (178)
Jacksonville St has a 64.15% Chance of Winning (-178)
Bryant vs Jacksonville St
Bryant has a 37.74% Chance of Winning (165)
Jacksonville St has a 62.26% Chance of Winning (-165)
Jacksonville St vs Wright St
Jacksonville St has a 47.17% Chance of Winning (112)
Wright St has a 52.83% Chance of Winning (-112)
TAM 

Delaware has a 18.87% Chance of Winning (430)
Villanova has a 81.13% Chance of Winning (-430)
Bryant vs Villanova
Bryant has a 11.32% Chance of Winning (783)
Villanova has a 88.68% Chance of Winning (-783)
Villanova vs Wright St
Villanova has a 88.68% Chance of Winning (-783)
Wright St has a 11.32% Chance of Winning (783)
Illinois vs Tennessee
Illinois has a 50.94% Chance of Winning (-103)
Tennessee has a 49.06% Chance of Winning (103)
Houston vs Tennessee
Houston has a 37.74% Chance of Winning (165)
Tennessee has a 62.26% Chance of Winning (-165)
Colorado St vs Tennessee
Colorado St has a 45.28% Chance of Winning (120)
Tennessee has a 54.72% Chance of Winning (-120)
Ohio St vs Tennessee
Ohio St has a 37.74% Chance of Winning (165)
Tennessee has a 62.26% Chance of Winning (-165)
Seton Hall vs Tennessee
Seton Hall has a 35.85% Chance of Winning (178)
Tennessee has a 64.15% Chance of Winning (-178)
TCU vs Tennessee
TCU has a 39.62% Chance of Winning (152)
Tennessee has a 60.38% Chance of

Loyola-Chicago has a 54.72% Chance of Winning (-120)
Michigan has a 45.28% Chance of Winning (120)
Loyola-Chicago vs UAB
Loyola-Chicago has a 56.6% Chance of Winning (-130)
UAB has a 43.4% Chance of Winning (130)
Chattanooga vs Loyola-Chicago
Chattanooga has a 37.74% Chance of Winning (165)
Loyola-Chicago has a 62.26% Chance of Winning (-165)
Longwood vs Loyola-Chicago
Longwood has a 39.62% Chance of Winning (152)
Loyola-Chicago has a 60.38% Chance of Winning (-152)
Delaware vs Loyola-Chicago
Delaware has a 33.96% Chance of Winning (194)
Loyola-Chicago has a 66.04% Chance of Winning (-194)
Bryant vs Loyola-Chicago
Bryant has a 30.19% Chance of Winning (231)
Loyola-Chicago has a 69.81% Chance of Winning (-231)
Loyola-Chicago vs Wright St
Loyola-Chicago has a 67.92% Chance of Winning (-211)
Wright St has a 32.08% Chance of Winning (211)
Michigan vs UAB
Michigan has a 60.38% Chance of Winning (-152)
UAB has a 39.62% Chance of Winning (152)
Chattanooga vs Michigan
Chattanooga has a 39.62% 

In [41]:
prediction_df = pd.DataFrame(predictions)

In [42]:
prediction_df.to_csv('2022_bracket_knn_predictions.csv', index=False)